In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from env import github_token, github_username
import unicodedata
import nltk
import prepare as p
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# CodeUp-DS-NLP-Project
 
### Project Goals 
* The goal of this classification project is to first identify key words of the programming language and create a machine learning model that can effectly predict the programming language used.
### The Plan
* Aquire ReadMe data from GitHub repositories via webscraping.
* Prepare data for exploration by:
    * Convert text to all lower case for normalcy.
    * Remove any accented characters, non-ASCII characters.
    * Remove special characters.
    * Lemmatize the words.
    * Remove stopwords.
    * Store the clean text and the original text for use in future notebooks.
#### Explore data in search of key features with the basic following questions:
* What are the most common words in READMEs?
* Does the length of the README vary by programming language?
* Do different programming languages use a different number of unique words?
* Are there any words that uniquely identify a programming language?
#### Develop a Model to predict happiness score
* Use key words identified to build predictive models of different types
* Evaluate models on train and validate data samples
* Select the best model based on accuracy
* Evaluate the best model on test data samples
#### Draw conclusions

### Steps to Reproduce
* Clone this repo.
* Acquire the data from GitHub
* Put the data in the file containing the cloned repo.
* Run notebook
### Conclusions
* 
#### Key TakeAway:
*
### Recommendations
*

In [2]:

#url = "https://github.com/search?3&q=stars%3A%3E0&s=stars&type=Repositories"
#reqs = requests.get(url)
#soup = BeautifulSoup(reqs.text, 'html.parser')
#urls = []
#for link in soup.find_all('a',class_="v-align-middle"):
#    print(link.get('href'))

In [3]:
# remember the lesson that Rosy showed you
#for i in range(1,50):
#    print(i)
#    time.sleep(10)

In [4]:

#url = "https://github.com/search?3&q=stars%3A%3E0&s=stars&type=Repositories"
#reqs = requests.get(url)
#soup = BeautifulSoup(reqs.text, 'html.parser')
#urls = []
#for link in soup.find_all('a',class_="v-align-middle"):
#    urls.append(link.get('href'))

In [5]:
#urls_repo = pd.read_csv('urls.csv', index_col=0)
#urls_repo['0']

In [6]:
REPOS = urls_repo['0']

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


#if __name__ == "__main__":
#    data = scrape_github_data()
#    json.dump(data, open("data.json", "w"), indent=1)

NameError: name 'urls_repo' is not defined

In [ ]:
#urls_df = scrape_github_data()

In [7]:
#df=pd.DataFrame(urls_df)
df = pd.read_csv('readme_df.csv')
df

,repo,language,readme_contents
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner](https://s3.amaz...
1,getify/You-Dont-Know-JS,NaN,# You Don't Know JS Yet (book series) - 2nd Ed...
2,microsoft/vscode,TypeScript,"# Visual Studio Code - Open Source (""Code - OS..."
3,electron/electron,C++,[![Electron Logo](https://electronjs.org/image...
4,axios/axios,JavaScript,"<h1 align=""center"">\n <b>\n <a href=""..."
...,...,...,...
847,nodejs/docker-node,Dockerfile,# Node.js\n\n[![dockeri.co](http://dockeri.co/...
848,cloudflare/cfssl,Go,# CFSSL\n\n[![Build Status](https://travis-ci....
849,leeoniya/uPlot,JavaScript,## 📈 μPlot\n\nA small ([~40 KB min](https://gi...
850,score-spec/spec,NaN,# Score\n\n![Score banner](/docs/images/banner...


In [8]:
df.dropna(inplace=True)

In [9]:
df.shape

(768, 3)

In [10]:
df = df[(df.language == 'Java') | (df.language=='JavaScript') | (df.language=='Python') | (df.language=='TypeScript')]

In [11]:
df.reset_index(drop = True, inplace= True)

In [12]:
def basic_clean(string):
    '''
    This function takes in a string and
    returns the string normalized.
    '''
    # we will normalize our data into standard NFKD unicode, feed it into an ascii encoding
    # decode it back into UTF-8
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    # utilize our regex substitution to remove our undesirable characters, then lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
    return string

In [13]:
#inshort_df[‘clean_text’] = inshort_df.content.apply(clean).apply(' ’.join)

In [14]:
df['clean_text']= df.readme_contents.apply(p.basic_clean)

### RATIOS

In [54]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels


,n,percent
JavaScript,169,0.448276
Python,88,0.233422
Java,62,0.164456
TypeScript,58,0.153846


In [56]:
def show_counts_and_ratios(df, column):
    """
    Takes in a dataframe and a string of a single column
    Returns a dataframe with absolute value counts and percentage value counts
    """
    labels = pd.concat([df[column].value_counts(),
                    df[column].value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    labels
    return labels




In [16]:
def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # make our tokenizer, taken from nltk's ToktokTokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # apply our tokenizer's tokenization to the string being input, ensure it returns a string
    string = tokenizer.tokenize(string, return_str = True)
    
    return string

In [17]:
df['clean_text']= df.clean_text.apply(p.tokenize)

In [18]:
df

,repo,language,readme_contents,clean_text
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner](https://s3.amaz...,freecodecamp social bannerhttpss3amazonawscomf...
1,microsoft/vscode,TypeScript,"# Visual Studio Code - Open Source (""Code - OS...",visual studio code open source code oss\nfeatu...
2,axios/axios,JavaScript,"<h1 align=""center"">\n <b>\n <a href=""...",h1 aligncenter\n b\n a hrefhttpsaxioshttpcomim...
3,microsoft/TypeScript,TypeScript,\r\n# TypeScript\r\n\r\n[![GitHub Actions CI](...,typescript\r\n\r\ngithub actions cihttpsgithub...
4,tensorflow/models,Python,"<div align=""center"">\n <img src=""https://stor...",div aligncenter\n img srchttpsstoragegoogleapi...
...,...,...,...,...
372,jquery-form/form,JavaScript,# jQuery Form [![Build Status](https://travis-...,jquery form build statushttpstravisciorgjquery...
373,Mailtrain-org/mailtrain,JavaScript,# Mailtrain v2\n\nMailtrain is a self hosted n...,mailtrain v2\n\nmailtrain is a self hosted new...
374,google/error-prone,Java,# Error Prone\n\nError Prone is a static analy...,error prone\n\nerror prone is a static analysi...
375,pentaho/pentaho-kettle,Java,# Pentaho Data Integration # \n\nPentaho Data ...,pentaho data integration \n\npentaho data inte...


In [19]:
def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # create our lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    # use a list comprehension to lemmatize each word
    # string.split() => output a list of every token inside of the document
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    # glue the lemmas back together by the strings we split on
    string = ' '.join(lemmas)
    #return the altered document
    return string

In [20]:
df['clean_text']= df.clean_text.apply(p.lemmatize)

In [21]:
def remove_stopwords(string, extra_words = [], exclude_words = []):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # assign our stopwords from nltk into stopword_list
    stopword_list = stopwords.words('english')
    # utilizing set casting, i will remove any excluded stopwords
    stopword_list = set(stopword_list) - set(exclude_words)
    # add in any extra words to my stopwords set using a union
    stopword_list = stopword_list.union(set(extra_words))
    # split our document by spaces
    words = string.split()
    # every word in our document, as long as that word is not in our stopwords
    filtered_words = [word for word in words if word not in stopword_list]
    # glue it back together with spaces, as it was so it shall be
    string_without_stopwords = ' '.join(filtered_words)
    # return the document back
    return string_without_stopwords

In [22]:
df['clean_text']= df.clean_text.apply(p.remove_stopwords)

In [23]:
df

,repo,language,readme_contents,clean_text
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner](https://s3.amaz...,freecodecamp social bannerhttpss3amazonawscomf...
1,microsoft/vscode,TypeScript,"# Visual Studio Code - Open Source (""Code - OS...",visual studio code open source code os feature...
2,axios/axios,JavaScript,"<h1 align=""center"">\n <b>\n <a href=""...",h1 aligncenter b hrefhttpsaxioshttpcomimg srch...
3,microsoft/TypeScript,TypeScript,\r\n# TypeScript\r\n\r\n[![GitHub Actions CI](...,typescript github action cihttpsgithubcommicro...
4,tensorflow/models,Python,"<div align=""center"">\n <img src=""https://stor...",div aligncenter img srchttpsstoragegoogleapisc...
...,...,...,...,...
372,jquery-form/form,JavaScript,# jQuery Form [![Build Status](https://travis-...,jquery form build statushttpstravisciorgjquery...
373,Mailtrain-org/mailtrain,JavaScript,# Mailtrain v2\n\nMailtrain is a self hosted n...,mailtrain v2 mailtrain self hosted newsletter ...
374,google/error-prone,Java,# Error Prone\n\nError Prone is a static analy...,error prone error prone static analysis tool j...
375,pentaho/pentaho-kettle,Java,# Pentaho Data Integration # \n\nPentaho Data ...,pentaho data integration pentaho data integrat...


In [24]:
def split_data(df, target):
    """"
    split_date takes in a dataframe  and target variable and splits into train , validate, test
    and stratifies on target variable
    The split is 20% test 80% train/validate. Then 30% of 80% validate and 70% of 80% train.
    Aproximately (train 56%, validate 24%, test 20%)
    returns train, validate, and test
    """
    # split test data from train/validate
    train_validate, test = train_test_split(df, test_size=.2,
                                        random_state=123,
                                        stratify=df[target])
    # split train from validate
    train, validate = train_test_split(train_validate, test_size=.3,
                                   random_state=123,
                                   stratify=train_validate[target])
    return train, validate, test

In [61]:
train, validate, test = split_data(df, 'language')

In [62]:
train.language.nunique()

4

In [63]:
# setting basic style parameters for matplotlib
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

In [75]:
JavaScript_words = ' '.join(train[train.language == 'JavaScript'].clean_text).split(' ')
Java_words = ' '.join(train[train.language == 'Java'].clean_text).split(' ')
Python_words = ' '.join(train[train.language == 'Python'].clean_text).split(' ')
TypeScript_words = ' '.join(train[train.language == 'TypeScript'].clean_text).split(' ')
All_words = basic_clean(' '.join(train.clean_text)).split(' ')

In [76]:
All_words

['pure',
 'purehttpscloudgithubusercontentcomassets44977952910991b554cca7b0311e4915753a12d91b34apngpure',
 'set',
 'small',
 'responsive',
 'cs',
 'module',
 'use',
 'every',
 'web',
 'project',
 'httpspurecssiopure',
 'build',
 'statushttpsgithubcompurecsspureactionsworkflowstestymlbadgesvg',
 'feature',
 'pure',
 'starting',
 'point',
 'every',
 'website',
 'web',
 'app',
 'take',
 'care',
 'cs',
 'work',
 'every',
 'site',
 'need',
 'without',
 'making',
 'look',
 'cookiecutter',
 'responsive',
 'grid',
 'customized',
 'need',
 'solid',
 'base',
 'built',
 'normalizecss',
 'fix',
 'crossbrowser',
 'compatibility',
 'issue',
 'consistently',
 'styled',
 'button',
 'work',
 'button',
 'element',
 'style',
 'vertical',
 'horizontal',
 'menu',
 'including',
 'support',
 'dropdown',
 'menu',
 'useful',
 'form',
 'alignment',
 'look',
 'great',
 'screen',
 'size',
 'various',
 'common',
 'table',
 'style',
 'extremely',
 'minimalist',
 'look',
 'supereasy',
 'customize',
 'responsive',
 '

In [77]:
JavaScript_freq = pd.Series(JavaScript_words).value_counts()
Java_freq = pd.Series(Java_words).value_counts()
Python_freq = pd.Series(Python_words).value_counts()
TypeScript_freq = pd.Series(TypeScript_words).value_counts()


In [78]:
JavaScript_freq

'                         3568
toptableofcontents         944
de                         823
test                       626
use                        524
                          ... 
supposed                     1
unnoticed                    1
node_tape_no_only_test       1
influenced                   1
routertrigger                1
Length: 20484, dtype: int64

In [79]:
word_counts = (pd.concat([JavaScript_freq, Java_freq, Python_freq, TypeScript_freq], axis=1, sort=True)
                .set_axis(['JavaScript', 'Java', 'Python', 'TypeScript'], axis=1, inplace=False)
                .fillna(0))

In [81]:
word_counts

,JavaScript,Java,Python,TypeScript
&#9;,467.0,179.0,6.0,50.0
',3568.0,283.0,750.0,526.0
0,113.0,16.0,44.0,39.0
00,0.0,0.0,36.0,0.0
000,3.0,0.0,1.0,0.0
...,...,...,...,...
zubehttpszubeiodocsapi,0.0,0.0,1.0,0.0
zuijiawoniuhttpsgithubcomzuijiawoniu,0.0,0.0,1.0,0.0
zwx6c000,0.0,0.0,1.0,0.0
zxvf,0.0,1.0,1.0,0.0
